**Answer 1**

In [89]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import PolynomialFeatures 
from sklearn import linear_model

In [90]:
gold = pd.read_csv("GOLD.csv")# Importing gold.csv

In [91]:
gold.head()

,Date,Price,Open,High,Low,Vol.,Change %,Pred,new
0,"May 04, 2017",28060,28400,28482,28025,0.08K,-1.79%,738.0,117.570740
1,"May 05, 2017",28184,28136,28382,28135,0.06K,0.44%,-146.0,295.430176
2,"May 08, 2017",28119,28145,28255,28097,7.85K,-0.23%,30.0,132.123714
3,"May 09, 2017",27981,28125,28192,27947,10.10K,-0.49%,357.0,101.298064
4,"May 10, 2017",28007,28060,28146,27981,9.28K,0.09%,124.0,112.153318


In [92]:
gold["Pred"].isnull().sum().sum() # Finding the numbers of null values in pred column

101

In [93]:
gold.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
Date        512 non-null object
Price       512 non-null int64
Open        512 non-null int64
High        512 non-null int64
Low         512 non-null int64
Vol.        512 non-null object
Change %    512 non-null object
Pred        411 non-null float64
new         512 non-null float64
dtypes: float64(2), int64(4), object(3)
memory usage: 36.1+ KB


In [94]:

regr = linear_model.LinearRegression() # Linear Regression model variable
pred_col = pd.DataFrame() # Making a new dataframe
new_col = pd.DataFrame() # Making a new dataframe

In [95]:
pred_col["Open"] = gold["Open"] # Assigning values to the new dataframe
pred_col["Price"] = gold["Price"]
pred_col["High"] = gold["High"]
pred_col["Low"]=gold["High"]
pred_col["Pred"] = gold["Pred"]

In [96]:
pred_col.head()

,Open,Price,High,Low,Pred
0,28400,28060,28482,28482,738.0
1,28136,28184,28382,28382,-146.0
2,28145,28119,28255,28255,30.0
3,28125,27981,28192,28192,357.0
4,28060,28007,28146,28146,124.0


In [97]:
pred_col.dropna(inplace=True) # Dropping the null values for linear regression

In [98]:
X=pred_col[["Price","Open","High","Low"]] # Setting X and Y variables for linear regression
Y=pred_col["Pred"]

In [132]:
# Performing linear regression on "pred" variable
regr.fit(X,Y) # Fitting the X and Y variables

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [100]:

print('Coefficients: \n', regr.coef_) # The coefficients of all the columns for Price,Open,High,Low

Coefficients: 
 [-1.03168728  0.46625087  0.29030895  0.29030895]


In [101]:
pred_data = pd.DataFrame()
pred_data = gold.loc[gold["Pred"].isnull()==True,["Price","Open","High","Low"]] # Extracting all the values which has null values in pred column 

In [102]:
a=pred_data['Price']
b=pred_data['Open']
c=pred_data['High']
d=pred_data['Low']
predicted_data = []
for i in range(len(a)):
    predicted_data.append(regr.predict([[a[i:i+1].item(),b[i:i+1].item(),c[i:i+1].item(),d[i:i+1].item()]]))#Storing the predicted data for the null pred values in the "predicted_data" list

In [103]:
gold.loc[gold["Pred"].isnull()==True,["Pred"]] = predicted_data # Completing the entire column in the Pred column in the main csv file

In [104]:
gold.tail() # Completed column

,Date,Price,Open,High,Low,Vol.,Change %,Pred,new
507,"Apr 26, 2019",31868,31851,31934,31705,9.67K,0.08%,269.136302,247.177322
508,"Apr 30, 2019",31625,31800,31824,31597,6.44K,-0.76%,432.770166,52.201158
509,"May 01, 2019",31563,31604,31657,31503,1.55K,-0.20%,329.578971,113.293305
510,"May 02, 2019",31203,31420,31425,31160,0.48K,-1.14%,448.268585,48.365693
511,"May 03, 2019",31341,31250,31500,31163,0.08K,0.44%,249.277190,429.924911


In [105]:
#Performing linear regression on "new" column
new_col["Open"] = gold["Open"]
new_col["Price"] = gold["Price"]
new_col["High"] = gold["High"]
new_col["Low"]=gold["High"]
new_col["new"] = gold["new"]

In [106]:
X1 = new_col[["Price","Open","High","Low"]]
Y1 = new_col[["new"]]

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=42)# Splitting the vaiables into training set and testing set

reg_new = linear_model.LinearRegression()
reg_new.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [108]:
predicted_sizes = reg_new.predict(X_test)  # Predicting values for test cases

In [109]:
print(r2_score(y_test,predicted_sizes)) #Printing the R2 score
print(np.sqrt(mean_squared_error(y_test,predicted_sizes)))

0.929733347074728
58.932357579042595


As we can see the R2 score is near to 0.92 which is near to 1. We can conclude the linear regression model fits the "new" column well.

In [110]:
#Performing Polynomial regression on "new" column
poly_reg = PolynomialFeatures(degree=4)
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=42)
X_poly_train = poly_reg.fit_transform(X_train)
X_poly_test = poly_reg.fit_transform(X_test)

In [111]:
reg_new.fit(X_poly_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [112]:
predicted_sizes = reg_new.predict(X_poly_test)

In [113]:
print(r2_score(y_test,predicted_sizes))
print(np.sqrt(mean_squared_error(y_test,predicted_sizes)))

0.9609682890558998
43.92259353050564


As we can see the r2 score is 0.96. This model fits the data very well. Polynomial regression is better than linear reression model which had values 0.92 for the "new" column

In [114]:
#Performing polynomial regression for "pred" column
poly_reg = PolynomialFeatures(degree=4)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_poly_train = poly_reg.fit_transform(X_train)
X_poly_test = poly_reg.fit_transform(X_test)
reg_new.fit(X_poly_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [115]:
predicted_sizes = reg_new.predict(X_poly_test)

In [116]:
print(r2_score(y_test,predicted_sizes))
print(np.sqrt(mean_squared_error(y_test,predicted_sizes)))

-1.5223848499733679
394.49507394517843


As we can see the r2 score is negative(=-1.52), the model does not fit the data.

As we can infer from the results polynomial regression does not fit the "pred" column but fits the "new" column very well. But linear regression fits both model well.

The "pred" column is linear function of variables and the ""new" column is polynomial function of variables

The sensitivity of a stock with respect to the market depends on its beta value.

When the market values jumps somehow the stock value varies according to its beta values 

**Answer 2**

In [117]:
lt = pd.read_csv("HDFC.csv")
nifty = pd.read_csv("Nifty50.csv")#Loading the Nifty50 index file
nifty.head()

,Date,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr)
0,15-May-2017,9433.55,9449.25,9423.10,9445.40,145163953,7790.97
1,16-May-2017,9461.00,9517.20,9456.35,9512.25,171709433,9209.89
2,17-May-2017,9517.60,9532.60,9486.10,9525.75,211856040,10726.23
3,18-May-2017,9453.20,9489.10,9418.10,9429.45,199340647,10782.46
4,19-May-2017,9469.90,9505.75,9390.75,9427.90,259861396,11544.77


In [118]:
lt = lt[lt["Series"]=="EQ"] # Removing row not having series value as EQ

In [119]:
# Setting date as Index for both stock and market data
lt["Date"] = pd.to_datetime(lt["Date"])
nifty["Date"] = pd.to_datetime(nifty["Date"])
nifty.set_index("Date",inplace = True)
lt.set_index("Date",inplace = True)

In [120]:
daily_returns_stock = lt['Close Price'].pct_change()# Calculating daily returns for stock data
monthly_returns_stock = lt['Close Price'].resample('M').ffill().pct_change()# Calculating monthly returns for stock data

In [121]:
lt.head()

,Symbol,Series,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
Date,,,,,,,,,,,,,,
2017-05-15,HDFC,EQ,1549.80,1554.5,1572.60,1554.10,1561.0,1559.50,1562.62,1270297,1.984990e+09,59917,955875,75.25
2017-05-16,HDFC,EQ,1559.50,1558.0,1569.00,1554.00,1568.0,1566.55,1564.05,2114918,3.307844e+09,79354,1652379,78.13
2017-05-17,HDFC,EQ,1566.55,1565.5,1570.95,1549.75,1550.1,1552.50,1557.08,2161434,3.365526e+09,80317,1794472,83.02
2017-05-18,HDFC,EQ,1552.50,1547.0,1558.60,1526.50,1545.0,1537.05,1539.03,2404372,3.700407e+09,85842,1837821,76.44
2017-05-19,HDFC,EQ,1537.05,1545.0,1548.00,1515.35,1526.5,1520.60,1526.14,2142433,3.269652e+09,113875,1601236,74.74


In [122]:
#Setting the first value as 0
monthly_returns_stock[0]=0
daily_returns_stock[0]=0

In [123]:
daily_returns_market = nifty['Close'].pct_change()#Calculating the daily return for nifty index
monthly_returns_market = nifty['Close'].resample('M').ffill().pct_change()#Calculating the monthly return for nifty index
daily_returns_market[0]=0
monthly_returns_market[0]=0

In [124]:
rs =daily_returns_stock.values.reshape(-1,1)# Assigning daily return stock values a variable
rm = daily_returns_market.values.reshape(-1,1)# Assigning daily return market values a vaiable

In [125]:
reg_b = linear_model.LinearRegression()
reg_b.fit(rm,rs) #Fitting the values for daily return 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [126]:
beta_daily = reg_b.coef_# Finding the slope 

In [127]:
beta_daily # This is our daily return beta value

array([[1.13003525]])

In [128]:
rs =monthly_returns_stock.values.reshape(-1,1)# Assigning monthly return stock values a variable
rm = monthly_returns_market.values.reshape(-1,1)# Assigning monthly return market values a vaiable

In [129]:
reg_b = linear_model.LinearRegression()
reg_b.fit(rm,rs)#Fitting the values for monthly return

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [130]:
beta_monthly=reg_b.coef_# Finding the slope 

In [131]:
beta_monthly# This is our monthly return beta value

array([[0.94520816]])

The daily beta value is greter than 1 i.e 1.13. This means stock values are more volatile the the market. The return would be 113% of what the market return on a daily basis.

The monthly beta value is less than 1 i.e. 0.94. This means the stock values are less volatile than the market.It would have returned only 94% of what the market returned on the monthly period.

A company with negative beta is negatively correlated to the returns of the market. For example a company with a beta of -0.2, which would have returned -2% when the market was up by 10%  